http://www.adeveloperdiary.com/data-science/machine-learning/derivation-and-implementation-of-baum-welch-algorithm-for-hidden-markov-model/

In [1]:
%pylab inline
plt.style.use('./BG.mplstyle')
%config InlineBackend.figure_format = 'retina'

Duplicate key in file './BG.mplstyle', line 21 ('patch.linewidth: 0.5')
Duplicate key in file './BG.mplstyle', line 34 ('grid.linestyle: -    # solid line')


Populating the interactive namespace from numpy and matplotlib


In [2]:
# import pandas as pd
# import numpy as np

# def forward(V, a, b, initial_distribution): # V=observations, a=trans_p, b=emis_p, initial_dist=pi 
#     alpha = np.zeros((V.shape[0], a.shape[0])) #shape (n_rows_in_excel * n_hidden_states) Here (500,2) .n_examples (not sure if each is an observation, each is one line of the excel)
#     alpha[0, :] = initial_distribution * b[:, V[0]] # apply initial distribution * emission prob on (row zero, and all 2 columns)
#     #print(alpha[0, :]) #results in for example [0.05555556 0.08333333]
#     for t in range(1, V.shape[0]): # n_examples of csv 500
#         for j in range(a.shape[0]): # n_hidden_states 2
#             # Matrix Computation Steps
#             # ((1x2) . (1x2)) * (1)
#             # (1)             * (1)
#             alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
#             #alpha[example, state] = alpha[prev_example] * trans_p for that state * emis_p for that state, for that obs
 
#     return alpha
 
# def backward(V, a, b):
#     beta = np.zeros((V.shape[0], a.shape[0]))
 
#     # setting beta(T) = 1
#     beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
#     # Loop in backward way from T-1 to
#     # Due to python indexing the actual loop will be T-2 to 0
#     for t in range(V.shape[0] - 2, -1, -1):
#         for j in range(a.shape[0]):
#             beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
 
#     return beta
 
# def baum_welch(V, a, b, initial_distribution, n_iter=100):
#     M = a.shape[0] # maximization step. shape n_hidden_states. Here 2
#     T = len(V) # len of observations
 
#     for n in range(n_iter): #for each epoch
#         alpha = forward(V, a, b, initial_distribution)
#         beta = backward(V, a, b)
 
#         xi = np.zeros((M, M, T - 1)) #here (2 x 2 x 500)>(n_hid, n_hid, n_observ)
#         for t in range(T - 1): #for all obs except last (since you need to use future time stamp)
#             denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
#             for i in range(M):
#                 numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
#                 xi[i, :, t] = numerator / denominator
 
#         gamma = np.sum(xi, axis=1)
#         a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
#         # Add additional T'th element in gamma
#         gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
#         K = b.shape[1] #n_of_unique_visible_states. Here 3, in my case n of unique DAs. used to calculate emission of visible state (DA_0 to DA_n) given a hidden state
#         denominator = np.sum(gamma, axis=1)
#         for l in range(K):
#             b[:, l] = np.sum(gamma[:, V == l], axis=1) #sum all row values from gamma only from column of that DA
 
#         b = np.divide(b, denominator.reshape((-1, 1)))
 
#     return {"a":a, "b":b}
 
    
# data = pd.read_csv('data_python.csv')
 
# V = data['Visible'].values # each row of csv corresponding to observations
 
# # Transition Probabilities >>>>>> shape (N_hidden_states * N_hidden_states)
# a = np.ones((2, 2))
# a = a / np.sum(a, axis=1)
 
# # Emission Probabilities >>>>>> # shape (rows = N_hidden_states, columns = N_DAs). (prob of given a state observing a specific DA)
# b = np.array(((1, 3, 5), (2, 4, 6)))
# print(b)
# b = b / np.sum(b, axis=1).reshape((-1, 1))
# print(b)
 
# # Probabilities for the initial distribution
# initial_distribution = np.array((0.5, 0.5))
 
# print(baum_welch(V, a, b, initial_distribution, n_iter=100))

## applied to my case

In [3]:
import pickle
import pandas as pd

a_file = open("./src/generated_files/input_hmm.pkl", "rb")
input_hmm = pickle.load(a_file)

unigram = input_hmm.get('unigram')
bi_fut = input_hmm.get('bi_fut')
bi_past = input_hmm.get('bi_past')
uni = input_hmm.get('uni')

df = pd.read_csv('./data_TM2/processed/processed_utterances_sentence_DA_labeling.csv', index_col=0)

In [4]:
def obs_as_int(uni, unigram):
    '''returns unique list with all observations encoded into numbers.
    observation encodings are in the same order as the array weights used as input to the emission prob'''
    
    unique_DA = list(uni.keys())
    da_to_int = {}

    for i in unique_DA:
        da_to_int[i] = unique_DA.index(i)

    observation_as_int = []

    for obs in unigram:
        for da in obs:
            for k in da_to_int.keys():
                if da == k:
                    da = da_to_int.get(k)
            observation_as_int.append(da)  
    return observation_as_int, unique_DA

observation_as_int, unique_DA = obs_as_int(uni, unigram)

In [ ]:
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

def forward(V, a, b, initial_distribution): # V=observations, a=trans_p, b=emis_p, initial_dist=pi 
    alpha = np.zeros((V.shape[0], a.shape[0])) #shape (n_rows_in_excel * n_hidden_states) Here (500,2) .n_examples (not sure if each is an observation, each is one line of the excel)
    alpha[0, :] = initial_distribution * b[:, V[0]] # apply initial distribution * emission prob on (row 0, and all 2 columns)
    #print(alpha[0, :]) #results in for example [0.05555556 0.08333333]
    for t in range(1, V.shape[0]): # n_examples of csv 500
        for j in range(a.shape[0]): # n_hidden_states 2
            # Matrix Computation Steps
            # ((1x2) . (1x2)) * (1)
            # (1)             * (1)
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
            #alpha[example, state] = alpha[prev_example] * trans_p for that state * emis_p for that state, for that obs
 
    return alpha
 
def backward(V, a, b):
    beta = np.zeros((V.shape[0], a.shape[0]))
 
    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((a.shape[0]))
 
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
 
    return beta
 
def baum_welch(V, a, b, initial_distribution, n_iter=100):
    M = a.shape[0] # maximization step. shape n_hidden_states >>>> Here 2
    T = len(V) # len of observations >>> Here 635219
 
    for n in range(n_iter): #for each epoch
        alpha = forward(V, a, b, initial_distribution)
        beta = backward(V, a, b)
 
        xi = np.zeros((M, M, T - 1)) #here (2 x 2 x 500)>(n_hid, n_hid, n_observ)
        for t in range(T - 1): #for all obs except last (since you need to use future time stamp)
            denominator = np.dot(np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator
 
        gamma = np.sum(xi, axis=1)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))
 
        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))
 
        K = b.shape[1] #n_of_unique_visible_states. Here 3, in my case n of unique DAs. used to calculate emission of visible state (DA_0 to DA_n) given a hidden state
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1) #sum all row values from gamma only from column of that DA
 
        b = np.divide(b, denominator.reshape((-1, 1)))
 
    return {"a":a, "b":b}
 
V = np.array(observation_as_int)# shape (635219,) # each row of csv corresponding to observations
# V = np.array(observation_as_int[0:49]) #one example only
# V = np.array(observation_as_int[0:100]) #testing with 2 observations
 
# Transition Probabilities >>>>>> shape (N_hidden_states * N_hidden_states)
a = np.array(((0.1, 0.9), (0.6,0.4))) #if I want to input my own perception as initial state. each row (hidden state) sum to 1
 
# Emission Probabilities >>>>>> # shape (rows = N_hidden_states, columns = N_DAs). (prob of given a state observing a specific DA)
# b = np.array((list(uni.values()), list(uni.values()))) #using unigram probabilities. each row (hidden state) sum to 1  
b = np.ones((2, len(list(uni.values())))) #starts with ones
b = b / np.sum(b, axis=1).reshape((-1, 1)) #normalized per row
    
# Probabilities for the initial distribution
initial_distribution = np.array((0.9999, 0.0001))
 
bw = baum_welch(V, a, b, initial_distribution, n_iter=50)


In [ ]:
def input_in_vit_BW_results(bw, initial_p):
    
    states = ['New', 'Current']
    start_p = {'New':initial_p[0],'Current':initial_p[1]}
    trans_p = {'New': {'New':bw.get('a')[0][0],'Current':bw.get('a')[0][1]}, 'Current': {'New': bw.get('a')[1][0],'Current':bw.get('a')[1][1]}} #the higher current, more it appears. 0.6 too litle, 0.7 too much
    emis_p = {}
    b_new = {}
    for da, be in zip(unique_DA, bw.get('b')[0]):
        b_new[da] = be
        
    b_cur = {}
    for da, be in zip(unique_DA, bw.get('b')[1]):
        b_cur[da] = be
    
    emis_p['New'] = b_new
    emis_p['Current'] = b_cur
    
    return states, start_p, trans_p, emis_p

states, start_p, trans_p, emis_p = input_in_vit_BW_results(bw, initial_distribution)


In [ ]:
def viterbi(obs, states, start_p, trans_p, emis_p): 
    ''' #code based on : https://en.wikipedia.org/wiki/Viterbi_algorithm#Pseudocode
    Code should receive emition prob of uni, bi and trigrams
    This also requires as input sentences (i.e: obs) with padding=2 on each side'''

    V = [{}]

    for st in states:
        V[0] [st] = {"prob": start_p[st] * emis_p[st] [obs[0]], "prev": None} #### OLD
        V.append({})
        V[1] [st] = {"prob": start_p[st] * emis_p[st] [obs[1]], "prev": None} #### NEW


    # Run Viterbi when t > 1
    for t in range(2, len(obs)-2): 
        V.append({})
        for st in states: #New and Current            
            max_tr_prob = V[t - 1] [states[0]] ["prob"] * trans_p[states[0]] [st] #0.026402178674778336*0.1 and same*0.9 ######### NEW
            prev_st_selected = states[0] 
            
            for prev_st in states[1:]: #(states[1:]) >>> Current
                tr_prob = V[t - 1] [prev_st] ["prob"] * trans_p[prev_st] [st] # a diferenca aqui em relacao ao max_tr_prob é que aqui se calcula o previous state
                if tr_prob > max_tr_prob: #if other state is higher than state 0 (i.e if Current > New)
                    max_tr_prob = tr_prob #then new max is Current instead on New. Here we store the actual probability
                    prev_st_selected = prev_st #and previous state is updated. Here we store name of state

            max_prob = max_tr_prob * emis_p[st] [obs[t]] 
            
            V[t] [st] = {"prob": max_prob, "prev": prev_st_selected}

    V = V[:-1]
    # for line in dptable(V):
       # print(line)

    optimal = []
    max_prob = 0.0
    best_st = None
    
    # Get most probable state and its backtrack
    for st, data in V[-1].items(): # V[-1] is the last dic. ex: {'New': {'prob': 6.983898154776834e-19, 'prev': 'Current'}, 'Current': {'prob': 1.5713770848247866e-18, 'prev': 'New'}
        if data["prob"] > max_prob: #6.983898154776834e-19 > 0.0 (in first step)
            max_prob = data["prob"] #update max prob to compare with second state in next iteration
            best_st = st
    optimal.append(best_st) #appending all best states
    previous = best_st #updating previous as best state to be used later


    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1): # start=len(V)-2, stop=-1, step=-1. Calculates from last step to first ###### NEW
        optimal.insert(0, V[t + 1] [previous] ["prev"])
        previous = V[t + 1] [previous] ["prev"]
    optimal = optimal[1:]
    
    # print ("The steps of states are " + " ".join(optimal) + " with highest probability of %s" % max_prob)
    return optimal, max_prob



def dptable(V):
    # Print a table of steps from dictionary
    yield " " * 5 + "     ".join(("%3d" % i) for i in range(len(V)))
    for state in V[0]:
        yield "%.7s: " % state + " ".join("%.7s" % ("%lf" % v[state] ["prob"]) for v in V)


In [ ]:
def all_dialog_hmm(unigram, bw, initial_distribution):
    states, start_p, trans_p, emis_p = input_in_vit_BW_results(bw, initial_distribution)

    hmm_seq = []
    count = 0
    for dialog in unigram:
        try:
            seq, p = viterbi(dialog, states, start_p, trans_p, emis_p)
            # hmm_seq.append([dialog[2:-1], seq, p]) #remove extra symbols from dialog (<s>, <ss>, <e>)
            hmm_seq.append([dialog[2:-2], seq[:-1], p]) #remove extra symbols from dialog (<s>, <ss>, <e>, <ee>)
        except Exception:
            count+=1
            pass

    return hmm_seq, count

In [ ]:
model_name = 'HMM'

unique_ids = input_hmm.get('unique_ids') # same as input_hmm['unique_ids']

hmm_seq, count = all_dialog_hmm(unigram, bw, initial_distribution)
print('Count of dialogues that failed and went to exception: ' + str(count))

# i = 10
# inspection_df = manual_inspection(i, df, hmm_seq, unique_ids)

# sorted_d, seq_sizes, seq_to_plot = sorted_seq_and_counts_hmm(hmm_seq)
# plot_literal_count_per_seq(seq_sizes, model_name)
# plot_len_grouped = plot_grouped_per_len_seq(sorted_d, seq_to_plot, model_name)
 